In [1]:
import os
import tensorflow as tf
import librosa
import numpy as np

2023-05-06 11:56:49.972995: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 11:56:50.039159: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 11:56:50.040174: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 11:56:51.190163: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
DATA_DIR = "audio_data"

In [3]:
label2id = {
    "ANG": 0,
    "NEU": 1,
    "HAP": 2,
    "FEA": 3,
    "DIS": 4,
    "SAD": 5
}

In [4]:
def prepare_audio_ds():

    data = []
    labels = []
    length = 3 * 16000
    for file in os.listdir("audio_data"):
        file_path = os.path.join("audio_data",file)
        audio, _ = librosa.load(file_path,sr=16000)
        if len(audio) < length:
            audio = np.pad(audio,(0,length-len(audio)),'constant')
        else:
            audio = audio[:length]

        label = label2id[file_path.split("_")[3]]
        melspec = librosa.feature.melspectrogram(y=audio, sr=16000)

        data.append(melspec)
        labels.append(label)

    data_ds = tf.data.Dataset.from_tensor_slices(data)
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((data_ds, label_ds))

In [5]:
def preprocess(x, label):

    inputs = processor.preprocess(x,do_rescale=False,do_resize=False,image_mean=0,image_std=0.5,return_tensors="tf") 
    return inputs["pixel_values"], label

def preprocess2(x):

    inputs = processor.preprocess(x,do_rescale=False,do_resize=False,image_mean=0,image_std=0.5,return_tensors="tf") 
    return inputs["pixel_values"]
    

In [12]:
def prepare_ssl_ds(num_classes=6, train_percent=0.8, label_percent = 0.5, batch_sizel=16, batch_sizeu=16):

    data = {i:[] for i in range(num_classes)}
    length = 3 * 16000
    for file in os.listdir("audio_data"):
        file_path = os.path.join("audio_data",file)
        audio, _ = librosa.load(file_path,sr=16000)
        if len(audio) < length:
            audio = np.pad(audio,(0,length-len(audio)),'constant')
        else:
            audio = audio[:length]

        label = label2id[file_path.split("_")[3]]
        melspec = librosa.feature.melspectrogram(y=audio, sr=16000,n_fft=512,hop_length=430,n_mels=112)[:,:,np.newaxis]

        data[label].append(melspec)

    labeled_split, unlabeled_split, test_split = ([],[]), [], ([],[])
    for label in range(num_classes):
        label_data = data[label]
        train_size = int(train_percent*len(label_data))
        test_size = len(label_data) - train_size
        labeled_size = int(label_percent * train_size)

        test_split[0].extend(label_data[train_size:])
        test_split[1].extend([label for _ in range(test_size)])
        
        labeled_split[0].extend(label_data[:labeled_size])
        labeled_split[1].extend([label for _ in range(labeled_size)])
    
        unlabeled_split.extend(label_data[labeled_size:train_size])

    labeled = tf.data.Dataset.from_tensor_slices(labeled_split)
    labeled = labeled.shuffle(len(labeled)).batch(batch_sizel)

    unlabeled = tf.data.Dataset.from_tensor_slices(unlabeled_split)
    unlabeled = unlabeled.shuffle(len(unlabeled)).batch(batch_sizeu)

    test = tf.data.Dataset.from_tensor_slices(test_split)
    test = test.shuffle(len(test)).batch(16)
    
    return labeled, unlabeled, test

In [ ]:
ds = get_ds()

In [13]:
labeled, unlabeled, test = prepare_ssl_ds()

In [14]:
next(iter(labeled))

2023-05-06 12:03:33.117409: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [2974]
	 [[{{node Placeholder/_1}}]]
2023-05-06 12:03:33.117704: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [2974]
	 [[{{node Placeholder/_1}}]]


(<tf.Tensor: shape=(16, 112, 112, 1), dtype=float32, numpy=
 array([[[[3.10835254e-04],
          [1.13181675e-04],
          [5.34751918e-04],
          ...,
          [0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00]],
 
         [[4.97518806e-04],
          [2.69889692e-03],
          [9.57946468e-04],
          ...,
          [0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00]],
 
         [[5.71993703e-04],
          [9.02745128e-03],
          [1.54509407e-03],
          ...,
          [0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00]],
 
         ...,
 
         [[4.78189293e-07],
          [9.04746571e-07],
          [1.06656319e-06],
          ...,
          [0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00]],
 
         [[4.55601793e-07],
          [7.85294219e-07],
          [4.92827098e-07],
          ...,
          [0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00

In [4]:
test_file = "/home/andrei/SSL/audio_data/1001_DFA_FEA_XX.wav"
length = 3 * 16000
audio, _ = librosa.load(test_file,sr=16000)

if len(audio) < length:
    audio = np.pad(audio,(0,length-len(audio)),'constant')
else:
    audio = audio[:length]

audio.shape

(48000,)

In [11]:
melspec = librosa.feature.melspectrogram(y=audio, sr=16000,n_fft=512,hop_length=430,n_mels=112)
melspec.shape

(112, 112)

In [16]:
from transformers import ViTImageProcessor, TFViTModel

/home/andrei/.local/share/virtualenvs/SSL-B1obpiFk/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')

In [33]:
model = TFViTModel.from_pretrained('google/vit-base-patch16-224')

Some layers from the model checkpoint at google/vit-base-patch16-224 were not used when initializing TFViTModel: ['classifier']
- This IS expected if you are initializing TFViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit/pooler/dense/bias:0', 'vit/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [107]:
norm_mel = melspec / np.max(melspec)

In [108]:
norm_mel = norm_mel[np.newaxis,:,:,np.newaxis]
norm_mel.shape

(1, 224, 224, 1)

In [18]:
batch = next(iter(labeled))

In [20]:
batch[0]

<tf.Tensor: shape=(16, 112, 112, 1), dtype=float32, numpy=
array([[[[1.3448650e-03],
         [1.6590778e-04],
         [1.8373945e-03],
         ...,
         [0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00]],

        [[2.0199262e-03],
         [1.2314869e-03],
         [2.0786505e-03],
         ...,
         [0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00]],

        [[2.3793723e-03],
         [1.2008757e-03],
         [1.6865426e-03],
         ...,
         [0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00]],

        ...,

        [[9.0859508e-08],
         [1.1403018e-07],
         [8.6782372e-08],
         ...,
         [0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00]],

        [[2.3369414e-07],
         [3.0757215e-07],
         [7.6529949e-07],
         ...,
         [0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00]],

        [[6.9941869e-08],
         [3.6381815e-07],
         [6.000

In [35]:
b = batch[0]/tf.reduce_max(batch[0])
b = tf.repeat(b,3,axis=-1)
inputs = processor.preprocess(b,image_mean=0,image_std=0.5,return_tensors="tf")

tf.reduce_max(inputs["pixel_values"])

<tf.Tensor: shape=(), dtype=float32, numpy=1.5137255>

In [34]:
model(**inputs)

TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor: shape=(16, 197, 768), dtype=float32, numpy=
array([[[ 0.02334747,  1.0715562 , -1.200588  , ..., -0.457747  ,
          0.21726893, -0.01203843],
        [ 0.4471927 ,  1.4031292 , -0.12195988, ...,  0.14339238,
          0.99487036,  0.18174091],
        [ 0.2698606 , -0.04117971, -0.03693089, ..., -0.63055205,
          1.6403565 ,  1.3969635 ],
        ...,
        [ 0.4800619 ,  0.7794227 , -0.08498895, ..., -0.74754244,
          0.84426796,  0.52911854],
        [ 0.42537072,  0.7792351 , -0.05819314, ..., -0.95499545,
          0.8533628 ,  0.4238273 ],
        [ 0.40322465,  0.9207795 , -0.2419951 , ...,  0.14645788,
          0.60421413, -0.28083032]],

       [[ 0.3101172 ,  0.31281754, -0.26379558, ..., -0.49732864,
          0.07916331,  0.03241296],
        [ 0.51191   ,  0.83087844,  0.6871592 , ..., -0.41650552,
          0.30407572,  0.12435514],
        [ 1.4944315 ,  0.5677866 ,  0.0409994 , ..., -0.6730571 ,
